# Here lies my attempt to deal with usgs stream gauge data and incorporate it into a map...

###### USGS Stream Gauge Site Information
* [USGS NWIS - IV REST - Info](http://waterservices.usgs.gov/rest/IV-Service.html)
* [USGS NWIS - IV REST - Test tool](http://waterservices.usgs.gov/rest/IV-Test-Tool.html)
* [And finally, their FAQ for automated retrievals](http://help.waterdata.usgs.gov/faq/automated-retrievals)
* [USGS NWIS Gage statistics](http://waterservices.usgs.gov/rest/Statistics-Service.html)
Example JSON request:
  * Raw Example from site:
        http://waterservices.usgs.gov/nwis/iv/?sites=08313000&period=P7D&format=json
  * Sample hu8 query:
        http://waterservices.usgs.gov/nwis/dv/?format=json,1.1&huc=18020129&startDT=2015-09-30&parameterCd=00060,00065&siteType=LK,ST
  * Final URL:
        http://waterservices.usgs.gov/nwis/dv/?format=json,1.1&huc=18020129&period=P7D&parameterCd=00060,00065&siteType=ST

In [7]:
import urllib2
import json
import string
import sys
import io
import os
from time import gmtime, strftime

Most of this code was stolen from [here](https://github.com/ghaff/whitewater-gauges/blob/master/usgs_populate.py)

In [18]:
#hu8list = [18020104,18020111,18020116,18020126,18020128,18020129,18020158,18020159,18020161,18020162,18020163,18040003,18040012,18040013,18050001]
hu8tmp = hu8list[18]
output = {}
fileout = open('gaugesall_iv.json', 'w')

In [23]:
hutmp = [18]

In [24]:
#for hu in range(1,18):
for hu in hutmp:
    if hu < 10:
        hucstring = "0" + str(hu)
    else:
        hucstring = str(hu)
    print "Working on getting hu"+hucstring+" json info yo..."
    dailyurl = 'http://waterservices.usgs.gov/nwis/iv/?format=json,1.1&huc='+hucstring+'&parameterCd=00060,00065&siteType=ST'
    req = urllib2.Request(dailyurl) 
    opener = urllib2.build_opener() 
    f = opener.open(req) 
    entry = json.loads(f.read())
    print "Loaded..."
    count = int (len(entry['value']['timeSeries']) - 1)
    while count >= 0:
        agaugenum = entry['value']['timeSeries'][count]['sourceInfo']['siteCode'][0]['value']
        asitename = entry['value']['timeSeries'][count]['sourceInfo']['siteName']
        alat = entry['value']['timeSeries'][count]['sourceInfo']['geoLocation']['geogLocation']['latitude']
        along = entry['value']['timeSeries'][count]['sourceInfo']['geoLocation']['geogLocation']['longitude']
        astatecode = entry['value']['timeSeries'][count]['sourceInfo']['siteProperty'][2]['value']
        acountycode = entry['value']['timeSeries'][count]['sourceInfo']['siteProperty'][3]['value']
        agauge = {
                      
            "sitename": asitename,
            "pos": [along, alat],
            "flow": 0,
            "height": 0,
            "timestamp": 0,
            "statecode": astatecode,
            "countycode": acountycode
            
        }
        output[agaugenum] = agauge
        count = count - 1

Working on getting hu18 json info yo...
Loaded...


In [25]:
for k in output:
    
    agaugestr = '{ "_id" : "' + k + '",'
    asitenamestr = ' "sitename" : "' + output[k]["sitename"] + '" ,'
    astatecodestr = ' "statecode" : "' + output[k]["statecode"] + '" ,'
    acountycodestr = ' "countycode" : "' + output[k]["countycode"] + '" ,'
    aposstr = ' "pos" : [' + str(output[k]["pos"][0]) + ', ' + str(output[k]["pos"][1]) + '] ,'
    aflowstr = ' "flow" : "' + str(output[k]["flow"]) + '" ,'
    aheightstr = ' "height" : "' + str(output[k]["height"]) + '" ,'
    atimestampstr = ' "timestamp" : "' + str(output[k]["timestamp"]) + '" }' 

    outstr= agaugestr + asitenamestr + astatecodestr + acountycodestr + aposstr  + aflowstr + aheightstr + atimestampstr

    print outstr
    
    fileout.write(outstr)
    fileout.write("\n")
fileout.close()

{ "_id" : "08072760", "sitename" : "Langham Ck at W Little York Rd nr Addicks, TX" , "statecode" : "48" , "countycode" : "48201" , "pos" : [-95.646612, 29.86717035] , "flow" : "0" , "height" : "0" , "timestamp" : "0" }
{ "_id" : "11055500", "sitename" : "PLUNGE C NR EAST HIGHLANDS CA" , "statecode" : "06" , "countycode" : "06071" , "pos" : [-117.141704, 34.11834458] , "flow" : "0" , "height" : "0" , "timestamp" : "0" }
{ "_id" : "14164900", "sitename" : "McKENZIE RIVER ABV HAYDEN BR, AT SPRINGFIELD,OR" , "statecode" : "41" , "countycode" : "41039" , "pos" : [-122.9645273, 44.0712365] , "flow" : "0" , "height" : "0" , "timestamp" : "0" }
{ "_id" : "02492360", "sitename" : "WEST HOBOLOCHITTO CREEK NR MCNEILL, MS" , "statecode" : "28" , "countycode" : "28109" , "pos" : [-89.6861111, 30.6622222] , "flow" : "0" , "height" : "0" , "timestamp" : "0" }
{ "_id" : "10296700", "sitename" : "W WALKER RV BLW TOPAZ CANAL DIV NR TOPAZ, CA" , "statecode" : "06" , "countycode" : "06051" , "pos" : [-119

In [9]:
for hu in hu8tmp:
    hu = str(hu)
    print "Working on getting hu"+hu+" json info yo..."
    dailyname = 'hu'+hu+'_dailydata.json'
    dailyurl = 'http://waterservices.usgs.gov/nwis/dv/?format=json,1.1&huc='+hu+'&period=P7D&parameterCd=00060,00065&siteType=ST'
    print dailyurl
    print dailyname
    req = urllib2.Request(dailyurl) 
    opener = urllib2.build_opener() 
    f = opener.open(req) 
    entry = json.loads(f.read())
    count = int (len(entry['value']['timeSeries']) - 1)
    while count >= 0:
        agaugenum = entry['value']['timeSeries'][count]['sourceInfo']['siteCode'][0]['value']
        asitename = entry['value']['timeSeries'][count]['sourceInfo']['siteName']
        alat = entry['value']['timeSeries'][count]['sourceInfo']['geoLocation']['geogLocation']['latitude']
        along = entry['value']['timeSeries'][count]['sourceInfo']['geoLocation']['geogLocation']['longitude']
        agauge = {"sitename": asitename,"pos": [along, alat],"flow": 0,"height": 0,"timestamp": 0,"hucode": hu}
        output[agaugenum] = agauge      
        count = count - 1
        print "Done with one | count: " + str(count)

Working on getting hu18020104 json info yo...
http://waterservices.usgs.gov/nwis/dv/?format=json,1.1&huc=18020104&period=P7D&parameterCd=00060,00065&siteType=ST
hu18020104_dailydata.json
Done with one | count: 19
Done with one | count: 18
Done with one | count: 17
Done with one | count: 16
Done with one | count: 15
Done with one | count: 14
Done with one | count: 13
Done with one | count: 12
Done with one | count: 11
Done with one | count: 10
Done with one | count: 9
Done with one | count: 8
Done with one | count: 7
Done with one | count: 6
Done with one | count: 5
Done with one | count: 4
Done with one | count: 3
Done with one | count: 2
Done with one | count: 1
Done with one | count: 0
Done with one | count: -1


In [10]:
print output.keys()

[u'11391050', u'11390500', u'11390425', u'11389350', u'11391000', u'11390660', u'11391002', u'11389390', u'11390480', u'11390672', u'11391100', u'11390890', u'11390655', u'11391020', u'11389470', u'11389500', u'11389000']


In [11]:
output['11391050']

{'flow': 0,
 'height': 0,
 'hucode': '18020104',
 'pos': [-121.6721868, 38.96628416],
 'sitename': u'SUTTER BYPASS NR NICOLAUS CA',
 'timestamp': 0}